### Using GroupBy for Basic NFL Data Analysis

This is the notebook I used for the Bonus video of Part 2 of some basic analysis of NFL games.

You can find the YouTube Video here: 
https://www.youtube.com/watch?v=iaH3mMmPRs0

The goal is to learn and understand how Pandas' GroupBy works, while progressing on NFL Analysis.
The data file used is from his github repo: NFLscrapR
https://github.com/ryurko/nflscrapR-data/tree/master/games_data/regular_season

Ram Narasimhan

Questions? Twitter: @ramnarasimhan

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
base = "https://raw.githubusercontent.com/ryurko/nflscrapR-data/master/games_data/regular_season/reg_games_"
year = 2018
coda = ".csv"
url = base + str(year) + coda

# All the Part 1 steps in one cell

In [4]:
df = pd.read_csv(url)

# There are some columns that we don't want. Let's drop those
drop_columns = ['type', 'state_of_game', 'game_url']

# axis = 1 is to tell Pandas to drop the columns.
# inplace = True drops them permanently
df.drop(columns=drop_columns, axis=1, inplace=True)

# We create two new columns in the data frame and store it. 
# This column can take negative values (if the home team lost) or positive values (if the home team won)
df['home_adv_score'] = df['home_score'] - df['away_score']

#THe following line creates a new column, with True/False values for whether the home team won
df['home_win'] = df['home_adv_score'] >= 0 
df['away_win'] = df['home_adv_score'] < 0 

In [5]:
df #Our Data frame is now ready

,game_id,home_team,away_team,week,season,home_score,away_score,home_adv_score,home_win,away_win
0,2018090600,PHI,ATL,1,2018,18,12,6,True,False
1,2018090900,BAL,BUF,1,2018,47,3,44,True,False
2,2018090907,NYG,JAX,1,2018,15,20,-5,False,True
3,2018090906,NO,TB,1,2018,40,48,-8,False,True
4,2018090905,NE,HOU,1,2018,27,20,7,True,False
5,2018090904,MIN,SF,1,2018,24,16,8,True,False
6,2018090903,MIA,TEN,1,2018,27,20,7,True,False
7,2018090902,IND,CIN,1,2018,23,34,-11,False,True
8,2018090901,CLE,PIT,1,2018,21,21,0,True,False
9,2018090908,LAC,KC,1,2018,28,38,-10,False,True


Goal: The aim for our analysis is to calculate the Win-Loss record for each of the 32 NFL teams.

We do this using GroupBy. To help the computation, we've created 2 new columns above, called `home_win` and `away_win.` Both these columns are Boolean columns, containing True or False

# Using GroupBy

Take a grouping columns (home_team) and a numeric column of interest (home_win) and sum them to get the total home of home wins for each team

In [11]:
df.groupby('home_team')['home_win'].sum().head()

home_team
ARI    1.0
ATL    4.0
BAL    6.0
BUF    4.0
CAR    5.0
Name: home_win, dtype: float64

In [14]:
# TO get the NUMBER of Games each team has played in its home field, use .size()
df.groupby('home_team')['home_win'].size().head()

home_team
ARI    8
ATL    8
BAL    8
BUF    8
CAR    8
Name: home_win, dtype: int64

Instead of doing this one by one for each calculation, we can put them all inside one `agg()` function.

In [15]:
df.groupby('home_team')['home_win'].agg(hp=len, hw=sum).head()

,hp,hw
home_team,,
ARI,8,1.0
ATL,8,4.0
BAL,8,6.0
BUF,8,4.0
CAR,8,5.0


### Calculating Number of Losses. (Writing our own function)

One easy way to calculate the home losses is to take #Home games and subtract the Home_wins from it.
It is a 1-line function. Notice that this function takes a whole group (called `g` here) and then we calculate the size() minus sum().

In [16]:
def num_losses(g):
    return g.size - g.sum()

df.groupby('home_team')['home_win'].agg(hp=len, hw=sum, hl = num_losses)
df.groupby('away_team')['away_win'].agg(ap=len, aw=sum, al = num_losses)

,ap,aw,al
away_team,,,
ARI,8,2.0,6
ATL,8,3.0,5
BAL,8,4.0,4
BUF,8,2.0,6
CAR,8,2.0,6
CHI,8,5.0,3
CIN,8,2.0,6
CLE,8,2.0,6
DAL,8,3.0,5


### Putting it all together

Now that all our GroupBy calculations are done, let's store everything in a couple of useful data frames.

In [17]:
home_results = df.groupby('home_team')['home_win'].agg(hp=len, hw=sum, hl = num_losses)
away_results = df.groupby('away_team')['away_win'].agg(ap=len, aw=sum, al = num_losses)

In [19]:
print(home_results.head())
print(away_results.head())


           hp   hw  hl
home_team             
ARI         8  1.0   7
ATL         8  4.0   4
BAL         8  6.0   2
BUF         8  4.0   4
CAR         8  5.0   3
           ap   aw  al
away_team             
ARI         8  2.0   6
ATL         8  3.0   5
BAL         8  4.0   4
BUF         8  2.0   6
CAR         8  2.0   6


### Using `pd.merge()` to combine the two data frames into one wide data frame

In [36]:
team_results = pd.merge(home_results, away_results, left_index=True, right_index=True)

In [38]:
team_results.columns

Index(['hp', 'hw', 'hl', 'ap', 'aw', 'al'], dtype='object')

One final step is needed. We have home_wins `hw` and away_wins `aw`. Let's add them both to get the final result.

Two new columns:

In [39]:
team_results['Wins'] = team_results['hw'] + team_results['aw']
team_results['Losses'] = team_results['hl'] + team_results['al']

In [40]:
team_results

,hp,hw,hl,ap,aw,al,Wins,Losses
home_team,,,,,,,,
ARI,8,1.0,7,8,2.0,6,3.0,13
ATL,8,4.0,4,8,3.0,5,7.0,9
BAL,8,6.0,2,8,4.0,4,10.0,6
BUF,8,4.0,4,8,2.0,6,6.0,10
CAR,8,5.0,3,8,2.0,6,7.0,9
CHI,8,7.0,1,8,5.0,3,12.0,4
CIN,8,4.0,4,8,2.0,6,6.0,10
CLE,8,6.0,2,8,2.0,6,8.0,8
DAL,8,7.0,1,8,3.0,5,10.0,6


Our Goal is achieved. We now have Win-Loss record for each team for the 2018-19 NFL Season.